In [1]:
%load_ext autoreload
%autoreload 2
import io, os, re, logging, sys

sys.path.append("../")

import requests, tqdm, tarfile, itertools, html, time
from dateparser.date import DateDataParser
from dateparser import parse
from fuzzywuzzy import process, fuzz

import numpy as np
import pandas as pd
import dask.dataframe as dd

import torch
import torch.nn.functional as F
import pyro
import pyro.distributions as dist

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.nonparametric.smoothers_lowess import lowess

from src.visualization.visualize import visualize_shot

# Import Dataset

In [2]:
shots_df = pd.read_parquet("../data/raw/visualizercoffee_73202shots_2021-12-29_cleaned.parquet")
shots_df

,id,profile_title,user_id,drink_tds,drink_ey,espresso_enjoyment,bean_weight,drink_weight,grinder_model,grinder_setting,...,espresso_flow_weight_raw,espresso_temperature_mix,espresso_water_dispensed,espresso_temperature_goal,espresso_resistance_weight,espresso_temperature_basket,duration,user_name,bean_age,shot_time
0,14899484-a5f8-4ccc-b372-79700c8150ae,TurboBloom,8d827d10-510f-4475-bd54-63c93b275c16,0.0,0.0,80.0,15.0,35.2,Lagom P64,1.4,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[86.0, 86.67, 86.31, 82.06, 77.66, 74.73, 73.1...","[0.0, 0.092681884765625, 0.20140380859375, 0.3...","[86.0, 86.0, 86.0, 86.0, 86.0, 86.0, 86.0, 86....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[86.0, 85.12, 85.27, 85.49, 85.38, 85.35, 85.3...",NaN,None,NaN,18.493
1,76f936ee-5147-40ac-986e-f0f1be5ac97e,Best overall pressure profile,10f1e281-8fd0-4f43-94f6-d416713dabe0,0.0,0.0,0.0,18.0,36.1,Bentwood 63,120,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[88.0, 88.3, 85.79, 85.82, 86.44, 87.32, 88.0,...","[0.0, 0.09389038085937501, 0.18845825195312502...","[88.0, 88.0, 88.0, 88.0, 88.0, 88.0, 88.0, 88....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[88.0, 87.18, 87.39, 87.37, 87.31, 87.23, 87.0...",NaN,None,NaN,29.475
2,8853ae7c-5bf8-4194-93dd-462de63f1471,Default,30b3dd10-aebb-4f17-ba74-a1efcf2d51ea,0.0,0.0,0.0,NaN,40.3,None,None,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[90.5, 89.87, 87.2, 87.69, 87.95, 87.98, 88.18...","[0.0, 0.10127563476562501, 0.20233764648437502...","[90.5, 90.5, 90.5, 90.5, 90.5, 90.5, 90.5, 90....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[90.5, 87.93, 88.21, 88.23, 88.16, 88.14, 88.1...",NaN,None,NaN,35.054
3,a4d7a358-fc3f-451c-92f5-756adaa4a7dd,Rao Allongé,c8c4793c-6708-4e81-90e8-6d8a5b30ceee,0.0,0.0,0.0,NaN,124.5,Niche Zero,13.5,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[92.0, 91.63, 88.9, 87.29, 88.18, 89.33, 90.63...","[0.0, 0.094384765625, 0.19187011718750002, 0.2...","[92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[92.0, 90.0, 90.41, 90.36, 90.35, 90.44, 90.4,...",NaN,None,NaN,40.996
4,3ebcfff9-fb6f-47f6-a3df-affe01d55150,Londonium,db110ce3-77b7-4c92-9d18-1019c3841cc9,0.0,0.0,0.0,NaN,0.0,None,None,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[89.0, 89.64, 87.11, 87.89, 88.81, 89.41, 89.6...","[0.0, 0.09944458007812501, 0.21490478515625, 0...","[89.0, 89.0, 88.5, 88.5, 88.5, 88.5, 88.5, 88....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[89.0, 87.55, 87.81, 87.75, 87.76, 87.8, 87.74...",NaN,None,NaN,38.787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73197,a68afc41-645d-46c6-bfce-3dc9bf95ed0b,Default,b51718ab-bc11-463d-be90-5538921b05e4,0.0,0.0,0.0,0.0,0.0,None,None,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[94.0, 63.18, 63.69, 67.55, 73.02, 76.92, 79.8...","[0.0, 0.13038330078125002, 0.22129516601562502...","[94.0, 94.0, 94.0, 94.0, 94.0, 94.0, 94.0, 94....",None,"[94.0, 65.96, 65.81, 65.66, 65.48, 65.41, 65.2...",119.943,Christopher Wansing,NaN,119.943
73198,a37b6160-3e6a-4860-97cc-6b75a035c0da,Default,b51718ab-bc11-463d-be90-5538921b05e4,0.0,0.0,0.0,0.0,0.0,None,None,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[94.0, 88.57, 86.06, 86.0, 88.17, 89.77, 90.95...","[0.0, 0.055694580078125, 0.13795166015625002, ...","[94.0, 94.0, 94.0, 94.0, 94.0, 94.0, 94.0, 94....",None,"[94.0, 89.45, 89.35, 89.1, 88.84, 88.48, 88.2,...",119.739,Christopher Wansing,NaN,119.739
73199,3849dd2f-1678-488b-8f76-b166f53fe7ed,Gentle flat 2.5 ml per second,b51718ab-bc11-463d-be90-5538921b05e4,0.0,0.0,0.0,0.0,0.0,None,None,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[90.0, 91.46, 89.11, 87.98, 88.25, 89.31, 90.3...","[0.0, 0.049627685546875, 0.09853515625, 0.1473...","[90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90....",None,"[90.0, 90.7, 90.7, 90.6, 90.57, 90.51, 90.46, ...",82.796,Christopher Wansing,

In [3]:
shots_df.dtypes

id                                     object
profile_title                          object
user_id                                object
drink_tds                             float64
drink_ey                              float64
espresso_enjoyment                    float64
bean_weight                           float64
drink_weight                          float64
grinder_model                          object
grinder_setting                        object
bean_brand                             object
bean_type                              object
roast_date                     datetime64[ns]
espresso_notes                         object
roast_level                            object
bean_notes                             object
start_time                     datetime64[ns]
image_preview                          object
profile_url                            object
timeframe                              object
espresso_flow                          object
espresso_weight                   

In [4]:
shots_df.espresso_enjoyment

0        80.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
73197     0.0
73198     0.0
73199     0.0
73200     0.0
73201     0.0
Name: espresso_enjoyment, Length: 73202, dtype: float64

In [5]:
most_popular_grinders = shots_df["grinder_model"].value_counts()[:20]
most_popular_grinders

Niche Zero                  11059
Sette 270                    2389
Mahlkonig EK43               1930
Lagom P100                   1863
Bentwood 63                  1843
Lagom P64                    1709
Kafatek Monolith Flat        1534
Kafatek Monolith Max         1371
DF64                          850
Eureka Mignon Specialita      604
Weber EG-1                    432
Titus M3                      394
Baratza Forté                 392
Kinu M47 Simplicity           386
LeverCraft Ultra              306
Baratza Vario                 239
Mazzer Mini                   198
Eureka Atom Specialty 75      196
Macap M4                      187
Breville Smart Grinder        177
Name: grinder_model, dtype: int64